Carga de datos desde el sitio web de cdmx:

In [25]:
# Este conjunto de datos se puede descargar
# directamente desde Python.
from urllib.request import urlretrieve
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm import tqdm

# Se descargan todos los archivos excel para los años 1996-2023.
for year in tqdm(range(1996, 2023)):
    url = f"http://www.aire.cdmx.gob.mx/descargas/basesimeca/imeca{year}.xls"
    filename = f"../slides/data/{Path(url).name}"
    urlretrieve(url, filename)
    

  0%|          | 0/27 [00:00<?, ?it/s]

100%|██████████| 27/27 [00:56<00:00,  2.09s/it]


Carga de dataframe:

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm

dfs = []
for year in tqdm(range(1996, 2023)):
    df = pd.read_excel(f"../slides/data/imeca{year}.xls")
    dfs.append(df)

df_imeca = pd.concat(dfs, ignore_index=True)
df_imeca

100%|██████████| 27/27 [00:07<00:00,  3.63it/s]


,Fecha,Hora,Noroeste Ozono,Noroeste dióxido de azufre,Noroeste dióxido de nitrógeno,Noroeste monóxido de carbono,Noroeste PM10,Noreste Ozono,Noreste dióxido de azufre,Noreste dióxido de nitrógeno,...,Noroeste ozono,Noroeste PM25,Noreste ozono,Noreste PM25,Centro ozono,Centro PM25,Suroeste ozono,Suroeste PM25,Sureste ozono,Sureste PM25
0,1996-01-01,7,21.0,10,13,35,22,21.0,16,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1996-01-01,8,22.0,10,18,32,23,16.0,16,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1996-01-01,9,25.0,10,23,32,24,17.0,16,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1996-01-01,10,40.0,10,30,33,24,42.0,16,26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1996-01-01,11,58.0,10,11,33,24,66.0,16,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227011,2022-12-31,20,NaN,4,17,5,38,NaN,2,18,...,19.0,58.0,19.0,56.0,21.0,54.0,26.0,51.0,25.0,61.0
227012,2022-12-31,21,NaN,4,14,5,39,NaN,2,22,...,11.0,59.0,13.0,56.0,13.0,54.0,24.0,52.0,23.0,62.0
227013,2022-12-31,22,NaN,4,20,6,40,NaN,2,22,...,6.0,60.0,8.0,57.0,11.0,54.0,21.0,51.0,15.0,62.0
227014,2022-12-31,23,NaN,4,22,7,40,NaN,2,20,...,8.0,60.0,5.0,58.0,6.0,54.0,22.0,51.0,11.0,63.0


Vista preliminar:

In [3]:
df_imeca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227016 entries, 0 to 227015
Data columns (total 38 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   Fecha                          209472 non-null  datetime64[ns]
 1   Hora                           227016 non-null  int64         
 2   Noroeste Ozono                 193416 non-null  float64       
 3   Noroeste dióxido de azufre     227016 non-null  int64         
 4   Noroeste dióxido de nitrógeno  227016 non-null  int64         
 5   Noroeste monóxido de carbono   227016 non-null  int64         
 6   Noroeste PM10                  227016 non-null  int64         
 7   Noreste Ozono                  193416 non-null  float64       
 8   Noreste dióxido de azufre      227016 non-null  int64         
 9   Noreste dióxido de nitrógeno   227016 non-null  int64         
 10  Noreste monóxido de carbono    227016 non-null  int64         
 11  

Hay varios problemas: 

- La fecha está dividida en dos columnas

- Los nombres de las columnas no han sido capitalizados uniformemente.

- Los valores nulos acá están marcados con $-99$.


Corrección de capitalizaciones no uniformes:

In [4]:
df_imeca.Fecha = df_imeca.Fecha.combine_first(df_imeca.FECHA)

zonas = ["Noroeste", "Noreste", "Centro", "Suroeste", "Sureste"]
for zona in zonas:
    df_imeca[f"{zona} Ozono"] = df_imeca[f"{zona} Ozono"].combine_first(
        df_imeca[f"{zona} ozono"]
    )

df_imeca = df_imeca.drop(columns=["FECHA"] + [f"{zona} ozono" for zona in zonas])

Corrección de columna de fecha:

In [5]:
## Específicamos que la unidad es en horas al momento de realizar la conversión
df_imeca.Hora = pd.to_timedelta(df_imeca.Hora, unit='h')

df_imeca["Fecha-Hora"] = df_imeca.Fecha + df_imeca.Hora
## Esto sumará el valor de Fecha con el valor de Hora para cada índice.
df_imeca = df_imeca.drop(columns=["Fecha", "Hora"])
## Borramos columnas innecesarias de nuevo.

Marca de valores nulos:

In [6]:
df_imeca = df_imeca.replace(-99, np.nan)

Creación de categorias:

In [7]:
def clasificar_calidad(value):
    if value < 50:
        return 'Buena'
    elif 50 <= value < 100:
        return 'Mediocre'
    elif 100 <= value < 200:
        return 'Mala'
    else:
        return 'Peligrosa'

## Agrupación de columnas

for region in ['Sureste', 'Suroeste', 'Centro', 'Noroeste', 'Noreste']:
    columns = [i for i in df_imeca.columns if region in i]
    for i in columns:
        df_imeca[i + ' Calidad'] = df_imeca[i].apply(clasificar_calidad)

In [8]:
df_imeca[[i for i in df_imeca.columns if "Sureste" in i]]

,Sureste Ozono,Sureste dióxido de azufre,Sureste dióxido de nitrógeno,Sureste monóxido de carbono,Sureste PM10,Sureste PM25,Sureste Ozono Calidad,Sureste dióxido de azufre Calidad,Sureste dióxido de nitrógeno Calidad,Sureste monóxido de carbono Calidad,Sureste PM10 Calidad,Sureste PM25 Calidad
0,28.0,10.0,19.0,63.0,21.0,NaN,Buena,Buena,Buena,Mediocre,Buena,Peligrosa
1,37.0,10.0,18.0,72.0,22.0,NaN,Buena,Buena,Buena,Mediocre,Buena,Peligrosa
2,31.0,11.0,23.0,75.0,23.0,NaN,Buena,Buena,Buena,Mediocre,Buena,Peligrosa
3,42.0,11.0,27.0,77.0,22.0,NaN,Buena,Buena,Buena,Mediocre,Buena,Peligrosa
4,57.0,11.0,15.0,71.0,22.0,NaN,Mediocre,Buena,Buena,Mediocre,Buena,Peligrosa
...,...,...,...,...,...,...,...,...,...,...,...,...
227011,25.0,2.0,11.0,4.0,26.0,61.0,Buena,Buena,Buena,Buena,Buena,Mediocre
227012,23.0,2.0,11.0,4.0,25.0,62.0,Buena,Buena,Buena,Buena,Buena,Mediocre
227013,15.0,2.0,15.0,5.0,25.0,62.0,Buena,Buena,Buena,Buena,Buena,Mediocre
227014,11.0,2.0,17.0,5.0,28.0,63.0,Buena,Buena,Buena,Buena,Buena,Mediocre


Guardado después del procesamiento:

In [9]:
df_imeca.to_parquet("../slides/data/imeca1996_2022.parquet")
